In [24]:
import pandas as pd
import numpy as np
import gc
import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [2]:
invoice_stat = pd.read_csv("data/invoice_stat.csv")
order_info = pd.read_csv("data/order_info.csv")
tax_relate = pd.read_csv("data/tax_relate.csv")

orders = order_info[order_info["product_id"]=="c789941d88c84c6193be364ce18995f5"]
order_info["busi_time"] = pd.to_datetime(order_info["busi_time"], dayfirst=True)
invoice_client = set(invoice_stat["tax_code"])
tax_client = set(tax_relate["unifiedcredit_code"])
order_client = set(orders["tax_code"])
common_client = order_client.intersection(invoice_client).intersection(tax_client)

# orders preprocess
def remove_duplicates(df):
    if len(df) < 2:
        return df
    df = df.sort_values(by="created_on")
    passed = df[df["status"]=="RETRIAL_PASSED"]
    if len(passed) > 0:
        return passed.tail(1)
    else:
        return df.tail(1)
orders = orders.groupby("tax_code", as_index=False).apply(remove_duplicates)
orders = orders[orders["tax_code"].isin(common_client)].reset_index(drop=True)
orders["busi_time"] = pd.to_datetime(orders["busi_time"], dayfirst=True)
orders["busi_date"] = pd.to_datetime(orders.busi_time.apply(lambda x: x.replace(day=1).date()))

# invoice preprocess
invoice = invoice_stat[invoice_stat["tax_code"].isin(common_client)].reset_index(drop=True)
invoice = invoice[(~invoice["bill_month"].isnull()) & (invoice["bill_month"]!="0000-00")]
invoice.loc[:, "bill_month"] = pd.to_datetime(invoice["bill_month"].apply(lambda x: x + "-01"))
invoice_cols = ["price_tax", "price_amount", "tax_amount", "tax_rate", "invoice_qty", "company_qty",
                "price_tax_valid", "price_amount_valid", "tax_amount_valid", "tax_rate_valid", "invoice_qty_valid", "company_qty_valid"]
# tax preprocess
def parse_charge_project(x):
    if re.match(".*增值税.*", x):
        return "VAT"
    elif re.match(".*所得税.*", x):
        return "income_tax"
    else:
        return "other_tax"
tax = tax_relate[tax_relate["unifiedcredit_code"].isin(common_client)].reset_index(drop=True)
tax_cols = ["unifiedcredit_code", "pay_date", "charge_project_parse", "alreadypay_taxamount"]
tax["charge_project_parse"] = tax["charge_project"].apply(parse_charge_project)
tax["pay_date"] = pd.to_datetime(tax["payin_date"], dayfirst=True, errors="coerce")
tax["pay_date"] = pd.to_datetime(tax["pay_date"].apply(lambda x: x.replace(day=1).date()))
tax_stats = tax[tax_cols].groupby(["unifiedcredit_code", "pay_date", "charge_project_parse"], as_index=False).agg(sum)
tax_stats.rename({"unifiedcredit_code":"tax_code"}, axis=1, inplace=True)
arr = tax_stats.set_index(["tax_code", "pay_date", "charge_project_parse"]).unstack(2).reset_index().values
tax_stats = pd.DataFrame(arr, columns=["tax_code", "pay_date", "VAT", "income_tax", "other_tax"]).fillna(0)

del invoice_stat
del tax_relate
gc.collect()

C:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5,15,16,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0

In [3]:
def make_lag_df(ref):
    lag_df_dict = {}
    for lag in [3, 6, 9, 12]:
        lag_df_list = []
        for row in ref.iterrows():
            drange = pd.Series(pd.date_range(end=row[1]["busi_date"], periods=lag, freq="M", closed="right").date).apply(lambda x: x.replace(day=1))
            tax_code = pd.Series(np.repeat(row[1]["tax_code"], lag))
            tmp = pd.DataFrame({"tax_code":tax_code, "date": drange})
            tmp["date"] = pd.to_datetime(tmp["date"])
            lag_df_list.append(tmp)
        lag_df = pd.concat(lag_df_list)
        lag_df_dict[lag] = lag_df
    return lag_df_dict
lag_df_dict = make_lag_df(orders)

In [4]:
def make_invoice_lag_feature(stats: pd.DataFrame, lags: dict) -> pd.DataFrame:
    result = lags.get(3)
    result = result[["tax_code"]].drop_duplicates()
    for lag, lag_df in lags.items():
        df = lag_df.merge(stats, left_on=["tax_code", "date"], right_on=["tax_code", "bill_month"], how="left")
        df = df[["tax_code", "date"] + invoice_cols]
        df = df.groupby("tax_code", as_index=False)[["price_amount", "tax_amount", "invoice_qty", "company_qty",
                                                     "price_amount_valid", "company_qty_valid"]].\
                aggregate(price_amount=("price_amount", "sum"), tax_amount=("tax_amount", "sum"), invoice_qty=("invoice_qty", "sum"),
                    downstream_qty=("company_qty", "sum"), break_point_qty=("price_amount", "count"),
                    price_amount_valid=("price_amount_valid", "sum"), downstream_qty_valid=("company_qty_valid", "sum"))
        df["break_point_qty"] = lag - df["break_point_qty"]
        df["price_amount_valid_rate"] = df["price_amount_valid"] / df["price_amount"]
        df["downstream_qty_valid_rate"] = df["downstream_qty_valid"] / df["downstream_qty"]
        df.drop(["price_amount_valid", "downstream_qty_valid"], axis=1, inplace=True)
        if lag != 12:
            #
            yoy_df = lag_df.copy()
            yoy_df["date"] = yoy_df["date"].apply(lambda x: x.replace(year=x.year-1))
            yoy_df = yoy_df.merge(stats, left_on=["tax_code", "date"], right_on=["tax_code", "bill_month"], how="left")
            yoy_df = yoy_df[["tax_code", "date"] + invoice_cols]
            yoy_df = yoy_df.groupby("tax_code", as_index=False)[["price_amount", "tax_amount", "invoice_qty", "company_qty",
                                                                 "price_amount_valid", "company_qty_valid"]].\
                    aggregate(price_amount=("price_amount", "sum"), tax_amount=("tax_amount", "sum"), invoice_qty=("invoice_qty", "sum"),
                        downstream_qty=("company_qty", "sum"), break_point_qty=("price_amount", "count"),
                        price_amount_valid=("price_amount_valid", "sum"), downstream_qty_valid=("company_qty_valid", "sum"))
            yoy_df["price_amount_valid_rate"] = yoy_df["price_amount_valid"] / yoy_df["price_amount"]
            yoy_df["downstream_qty_valid_rate"] = yoy_df["downstream_qty_valid"] / yoy_df["downstream_qty"]
            yoy_df.drop(["price_amount_valid", "downstream_qty_valid"], axis=1, inplace=True)
            base = yoy_df.loc[:, ["price_amount", "invoice_qty", "downstream_qty", "price_amount_valid_rate", "downstream_qty_valid_rate"]]
            inc = (df.loc[:, ["price_amount", "invoice_qty", "downstream_qty", "price_amount_valid_rate" , "downstream_qty_valid_rate"]] - base).divide(base)
            inc.rename(columns={"price_amount":"yoy"+str(lag)+"_price_amount", "invoice_qty":"yoy"+str(lag)+"_invoice_qty", "downstream_qty":"yoy"+str(lag)+"_downstream_qty",
                                "price_amount_valid_rate":"yoy"+str(lag)+"_price_amount_valid_rate", "downstream_qty_valid_rate":"yoy"+str(lag)+"_downstream_qty_valid_rate"},
                    inplace=True)
            df = pd.concat([df, inc], axis=1)
            
            #
            mom_df = lag_df.copy()
            mom_df["date"] = mom_df["date"].apply(lambda x: x - relativedelta(months=lag))   
            mom_df = mom_df.merge(stats, left_on=["tax_code", "date"], right_on=["tax_code", "bill_month"], how="left")
            mom_df = mom_df[["tax_code", "date"] + invoice_cols]
            mom_df = mom_df.groupby("tax_code", as_index=False)[["price_amount", "tax_amount", "invoice_qty", "company_qty",
                                                                 "price_amount_valid", "company_qty_valid"]].\
                    aggregate(price_amount=("price_amount", "sum"), tax_amount=("tax_amount", "sum"), invoice_qty=("invoice_qty", "sum"),
                        downstream_qty=("company_qty", "sum"), break_point_qty=("price_amount", "count"),
                        price_amount_valid=("price_amount_valid", "sum"), downstream_qty_valid=("company_qty_valid", "sum"))
            mom_df["price_amount_valid_rate"] = mom_df["price_amount_valid"] / mom_df["price_amount"]
            mom_df["downstream_qty_valid_rate"] = mom_df["downstream_qty_valid"] / mom_df["downstream_qty"]
            mom_df.drop(["price_amount_valid", "downstream_qty_valid"], axis=1, inplace=True)
            base = mom_df.loc[:, ["price_amount", "invoice_qty", "downstream_qty", "price_amount_valid_rate", "downstream_qty_valid_rate"]]
            inc = (df.loc[:, ["price_amount", "invoice_qty", "downstream_qty", "price_amount_valid_rate", "downstream_qty_valid_rate"]] - base).divide(base)
            inc.rename(columns={"price_amount":"mom"+str(lag)+"_price_amount", "invoice_qty":"mom"+str(lag)+"_invoice_qty", "downstream_qty":"mom"+str(lag)+"_downstream_qty",
                                "price_amount_valid_rate":"mom"+str(lag)+"_price_amount_valid_rate", "downstream_qty_valid_rate":"mom"+str(lag)+"_downstream_qty_valid_rate"},
                    inplace=True)
            df = pd.concat([df, inc], axis=1)
            
            
        #renaming columns
        df.rename(columns={"price_amount":"lag"+str(lag)+"_price_amount", "tax_amount":"lag"+str(lag)+"_tax_amount", "invoice_qty":"lag"+str(lag)+"_invoice_qty",
                           "downstream_qty":"lag"+str(lag)+"_downstream_qty", "break_point_qty":"lag"+str(lag)+"_break_point_qty",
                           "price_amount_valid_rate":"lag"+str(lag)+"_price_amount_valid_rate", "downstream_qty_valid_rate":"lag"+str(lag)+"_downstream_qty_valid_rate"}, inplace=True)
        result = result.merge(df, on="tax_code")
    for col in result.columns:
        result.loc[(result[col]==np.inf) | (result[col]==-np.inf), col] = np.NaN
    return result

In [7]:
invoice_feature = make_invoice_lag_feature(invoice, lag_df_dict)

In [9]:
# tax features
def make_tax_lag_feature(stats: pd.DataFrame, lags: dict) -> pd.DataFrame:
    result = lags.get(3)
    result = result[["tax_code"]].drop_duplicates()
    for lag, lag_df in lags.items():
        df = lag_df.merge(stats, left_on=["tax_code", "date"], right_on=["tax_code", "pay_date"], how="left")
        df = df.groupby("tax_code", as_index=False)[["VAT", "income_tax", "other_tax"]].\
                aggregate(VAT=("VAT", "sum"), income_tax=("income_tax", "sum"), other_tax=("other_tax", "sum"))
        df.rename(columns={"VAT":"lag"+str(lag)+"_VAT", "income_tax":"lag"+str(lag)+"_income_tax",
                           "other_tax":"lag"+str(lag)+"_other_tax"}, inplace=True)
        result = result.merge(df, on="tax_code")
    return result


In [10]:
tax_feature = make_tax_lag_feature(tax_stats, lag_df_dict)

In [35]:
def make_order_feature(orders: pd.DataFrame, history: pd.DataFrame) -> pd.DataFrame:
    history_cpy = history[history["product_id"]!="c789941d88c84c6193be364ce18995f5"].copy()
    res = []
    for row in tqdm(list(orders[["tax_code", "busi_time"]].iterrows())):
        busi_time = row[1]["busi_time"]
        tax_code = row[1]["tax_code"]
        his = history_cpy.loc[
            (history_cpy["busi_time"]<busi_time) & (history_cpy["tax_code"]==tax_code), ["status", "money", "created_on"]
        ]
        his = remove_duplicates(his)
        cnt = len(his)
        passed = len(his[his["status"]=="RETRAIL_PASSED"])
        balance = his["money"].sum()
        tmp = pd.DataFrame({"tax_code":tax_code, "num_applied_products":cnt, "num_passed_products":passed, "loan_balance": balance}, index=[0])
        res.append(tmp)
    result = pd.concat(res).reset_index(drop=True)
    return result

In [36]:
order_feature = make_order_feature(orders, order_info)

100%|██████████| 8432/8432 [04:28<00:00, 31.40it/s]


In [46]:
dataset = order_feature.merge(invoice_feature, on="tax_code").merge(tax_feature, on="tax_code")
dataset = dataset.merge(orders[["tax_code", "status"]], on="tax_code")
dataset["status"] = dataset["status"].apply(lambda x: 1 if x=="RETRIAL_PASSED" else 0)
dataset.rename(columns={"status":"label"}, inplace=True)
dataset.to_csv("data/dataset_remodel.csv", index=False)